In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")
index_name = "course_homework"
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_homework'})

### Q2. Indexing the data

In [3]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/home/ahmedk/Courses/Bootcamps/llm-zoomcamp/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:05<00:00, 161.44it/s]


In [4]:
def elastic_search(query):

    search_query = {
        "size": 1,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }
    return es_client.search(index=index_name, body=search_query)

In [5]:
def find_scores(response):
    result_scores = []
    for hit in response['hits']['hits']:
        result_scores.append(hit["_score"])
    return result_scores

### Q3. Searching

In [6]:
query = "How do I execute a command in a running docker container?"
results = elastic_search(query)
print("Top ranking result score:", find_scores(results)[0])

Top ranking result score: 81.68475


### Q4. Filtering

In [7]:
def elastic_search(query):

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    return es_client.search(index=index_name, body=search_query)

In [8]:
response = elastic_search(query)
print(f"3rd question from the response", response['hits']['hits'][2]['_source']["question"])

3rd question from the response How do I copy files from a different folder into docker container’s working directory?


In [9]:
def get_results(response):
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit["_source"])
    return result_docs

### Q5. Building a prompt

In [10]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()

    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    context = "\n\n".join(
        context_template.format(
            question=doc['question'], text=doc['text'])for doc in search_results)
    return prompt_template.format(question=query, context=context)


In [11]:
response = elastic_search(query)
results = get_results(response)
prompt = build_prompt(query, results)
print("prompt length is:", len(prompt))

prompt length is: 1490


### Q6. Tokens

In [12]:
def llm(prompt):
    from openai import OpenAI

    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role':'user', 'content': prompt}],
    )
    return response.choices[0].message.content

In [13]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")
num_sent_tokens = len(encoding.encode(prompt))
print("Number of tokens in our prompt is ", num_sent_tokens)

Number of tokens in our prompt is  329


### Bonus: generating the answer (ungraded)

In [14]:
def rag(query):
    response = elastic_search(query)
    results = get_results(response)
    prompt = build_prompt(query, results)
    return llm(prompt)
openai_resp = rag(query)
num_recv_token = len(encoding.encode(openai_resp))
openai_resp

'To execute a command in a running Docker container, follow these steps:\n\n1. Use the `docker ps` command to list all running containers and find the container ID of the specific container you want to execute a command in.\n   ```sh\n   docker ps\n   ```\n\n2. Use the `docker exec -it` command followed by the container ID and the command you want to execute. For example, to start a bash session in the container, you would use:\n   ```sh\n   docker exec -it <container-id> bash\n   ```\n\nThese steps will allow you to run commands in your desired container.'

### Bonus: calculating the costs (ungraded)

In [15]:
avg_cost = (150*1000)*(0.005/1000) + (250*1000)*(0.015/1000)
f"Average cost is {avg_cost}$"

'Average cost is 4.5$'

In [16]:
q5_q6_cost = num_sent_tokens*(0.005/1000) + num_recv_token*(0.015/1000)
f"Homework cost is {q5_q6_cost}$"

'Homework cost is 0.00355$'